In [1]:
import torch, json
from math import floor
import pandas as pd
import numpy as np

from FileManager.dataManager import dataManager
from AnalyzeTools.models import autoregressive_integrated_moving_average, linear_regression, support_vector_regression, random_forest, gradient_boosting
from AnalyzeTools.prepare import data_split, model_eval, pathForSavingModels
from AnalyzeTools.preprocess import preprocessData
from AnalyzeTools.superModels import DEEPAR, TFT, RNN

c:\Users\zyf13\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 123


In [2]:
period = 'Day'
predict_type = 'single'
future_step = 7
test_size = 0.1
params_path = f'./Models'

product_object = json.load(open("./File information.json", "r", encoding='utf8'))

all_experiments= []
for product in product_object.keys():
    for raw_file_name in  product_object[product].keys():
        for product_type in product_object[product][raw_file_name]['product_types']:
            for target in product_object[product][raw_file_name]['targets']:
                all_experiments.append([product, raw_file_name, product_type, target])

In [4]:
n = 0
experiment = all_experiments[n]
product, raw_file_name, product_type, target = experiment
print(f"Product: {product}\nRaw file name: {raw_file_name}\nProduct_type: {product_type}\ntarget: {target}")

Product: pork
Raw file name: (중)경략가격집계 - 소,돼지
Product_type: 돼지 온도체
target: MAX_COST_AMT


In [5]:
df, product_and_product_type, product_attribute = dataManager(raw_file_name, product, product_type, target)

if len(df) == 0:
    raise ValueError("No data!")

df, input_features = preprocessData(df, 'date', target)

train_x_axis = df['date'][:-1*floor(len(df) * test_size)].values if type(test_size) == float else df['date'][:-1*test_size]
predictions_x_axis = df['date'][-1*floor(len(df) * test_size):].values if type(test_size) == float else df['date'][-1*test_size:]


-->Feature scores:
                 Features        Scores
6  DEFECT_MAX_COST_AMT  6.811574e+10
1         MIN_COST_AMT  1.222592e+04
5  DEFECT_MIN_COST_AMT  1.222443e+04
2         SUM_COST_AMT  4.538033e+02
7  DEFECT_SUM_COST_AMT  4.537454e+02
8    DEFECT_SUM_WEIGHT  2.279391e+01
3           SUM_WEIGHT  2.277102e+01
4           DEFECT_CNT  1.084715e+01
0                  CNT  1.083094e+01

-->TOP K features:
   ['MIN_COST_AMT', 'SUM_COST_AMT', 'DEFECT_MIN_COST_AMT', 'DEFECT_MAX_COST_AMT']

-->Final features:
  ['MIN_COST_AMT', 'SUM_COST_AMT']


In [6]:
# prepare dataset for statistics and Macnhine models
ml_split_params = {'Model': 'ML', 'Future': future_step}
X_train, X_test, y_train, y_test, input_scaler, output_scaler = data_split(df, input_features, target, test_size, scaling=True, **ml_split_params)

''' Input data into models and Evaluate model results '''
ml_searchCV_params = {
    'base_dir': params_path,
    'product_and_product_type': product_and_product_type,
    'attribute': product_attribute,
    'raw': raw_file_name,
    'predict_type': predict_type,
    'period': period,
    'step': future_step,
    'save': True
}
stdout = True
vis = True

X_train: (2940, 3) y_train: (2940,) X_test: (327, 3) y_test: (327,)


# Statistics

In [ ]:
print("\nARIMA")
arima_predictions = autoregressive_integrated_moving_average(y_train, y_test)


In [14]:
model_eval(y_test, arima_predictions, predictions_x_axis, stdout=stdout, vis=vis, **{'scaler': output_scaler})

MAPE: 0.033428485112001464 R square: 0.853928674963152


In [8]:
print("\nLinear Regression")
lr, _ = linear_regression(X_train, y_train)
lr_predictions = lr.predict(X_test)
model_eval(y_test, lr_predictions, predictions_x_axis, stdout=stdout, vis=vis, **{'scaler': output_scaler})


Linear Regression
MAPE: 0.07077922721389458 R square: 0.6254846676464798


# Machine learning

In [9]:
print("\nSupport Vector Regression")
svr, _ = support_vector_regression(X_train, y_train, search=True, **ml_searchCV_params)
svr_predictions = svr.predict(X_test)
model_eval(y_test, svr_predictions, predictions_x_axis, stdout=stdout, vis=vis, **{'scaler': output_scaler})


Support Vector Regression
--> Use the existed best parameters!

Best parameter for SVR is:
  {'C': 10, 'gamma': 0.01, 'epsilon': 0.005, 'kernel': 'rbf'}
MAPE: 0.08240021724510858 R square: 0.48896180239384246


In [15]:
print("\nRandom Forest")
rf, _ = random_forest(X_train, y_train, search=True, **ml_searchCV_params)
rf_predictions = rf.predict(X_test)
model_eval(y_test, rf_predictions, predictions_x_axis, stdout=stdout, vis=vis, **{'scaler': output_scaler})


Random Forest
--> Use the existed best parameters!

Best parameter for Random forest is:
  {'n_estimators': 150, 'max_depth': 5, 'max_features': 3, 'min_samples_leaf': 7, 'min_samples_split': 3}
MAPE: 0.09982951708851266 R square: 0.27628814550197967


In [11]:
print("\nGradient Boosting")
gb, _ = gradient_boosting(X_train, y_train, search=True, **ml_searchCV_params)
gb_predictions = gb.predict(X_test)
model_eval(y_test, gb_predictions, predictions_x_axis, stdout=stdout, vis=vis, **{'scaler': output_scaler})


Gradient Boosting
--> Use the existed best parameters!

Best parameter for Gradient Boosting is:
  {'n_estimators': 230, 'max_depth': 1, 'max_features': 3, 'min_samples_leaf': 7, 'min_samples_split': 9}
MAPE: 0.10483140977641749 R square: 0.14131198772482323
